In [73]:
import smtplib
import imaplib
import email
from bs4 import BeautifulSoup

In [187]:
mailer = imaplib.IMAP4_SSL('smtp.gmail.com')
mailer.login('adarsh.raigarha@gmail.com','Ady.OPJS')
mailer.select('inbox')

t, data = mailer.search(None, 'ALL')
#data - data is list of the email
email_ind = data[0]
# email_ing is the proper number of emails
indx = email_ind.split()
latest_mail_ind = indx[-1]
first_mail_ind = indx[0]
random_mail_ind = indx[30]

In [134]:
t1,data_mail = mailer.fetch(random_mail_ind, '(RFC822)')
data1 = data_mail[0] #this is first part of data mail
# print(len(data1))
# data[1]

msg = email.message_from_bytes(data1[1])
# print(msg['HTML'])
msg = msg.get_payload(1)

print(str(msg))

soup = BeautifulSoup(str(msg), features='lxml')

final_msg = soup.get_text(strip=True)

In [152]:
dict1 = {}
for i in range(1,1000):
    mail_ind = indx[i]
    t1,data_mail = mailer.fetch(mail_ind, '(RFC822)')
    data1 = data_mail[0] #this is first part of data_mail

    msg = email.message_from_bytes(data1[1])
    # print(msg['HTML'])
    if msg.is_multipart():
        try:            
            msg = msg.get_payload(i=0)
            soup = BeautifulSoup(str(msg), features='lxml')
            final_msg = soup.get_text(strip=True)
            dict1[i] = final_msg
        except:
            msg = msg.get_payload(i=1)
            soup = BeautifulSoup(str(msg), features='lxml')
            final_msg = soup.get_text(strip=True)
            dict1[i] = final_msg
    else:
        msg = msg.get_payload()
        soup = BeautifulSoup(str(msg), features='lxml')
        final_msg = soup.get_text(strip=True)
        dict1[i] = final_msg
        

In [154]:
len(dict1)

999

In [169]:
df_mails = pd.DataFrame(dict1.values(), columns=['content'])

In [173]:
spam_result_dict_mails = {}
for i in range(1,1000):
    spam_result_dict_mails[i] = 0

In [174]:
spam_result_DF_mails = pd.DataFrame(spam_result_dict_mails.values(), columns=['spam'])

In [181]:
right_mail = pd.concat([df_mails , spam_result_DF_mails['spam']], axis=1)

In [182]:
import csv

In [183]:
right_mail.to_csv('Mails_NotSpam.csv')

In [188]:
mailer.list()

('OK',
 [b'(\\HasNoChildren) "/" "INBOX"',
  b'(\\HasChildren \\Noselect) "/" "[Gmail]"',
  b'(\\All \\HasNoChildren) "/" "[Gmail]/All Mail"',
  b'(\\HasNoChildren \\Trash) "/" "[Gmail]/Bin"',
  b'(\\Drafts \\HasNoChildren) "/" "[Gmail]/Drafts"',
  b'(\\HasNoChildren \\Important) "/" "[Gmail]/Important"',
  b'(\\HasNoChildren \\Sent) "/" "[Gmail]/Sent Mail"',
  b'(\\HasNoChildren \\Junk) "/" "[Gmail]/Spam"',
  b'(\\Flagged \\HasNoChildren) "/" "[Gmail]/Starred"'])